In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# %cd /content/gdrive/My\ Drive/Colab_notebooks/RvIdrv

In [3]:
# !git clone https://github.com/boudinfl/pke.git

In [4]:
# !rm -r '/content/gdrive/My Drive/Colab_notebooks/RvIdrv/pke/pke/models/SemEval-2010_0.05_rbf_scale_False'

In [5]:
%cd /content/gdrive/My\ Drive/Colab_notebooks/RvIdrv/pke/

/content/gdrive/My Drive/Colab_notebooks/RvIdrv/pke


In [6]:
!pip install python-igraph

     |████████████████████████████████| 3.2MB 8.4MB/s 


In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import SnowballStemmer

import pke
from pke.unsupervised import NodeDegree
from pke.unsupervised import Betweeness
from pke.unsupervised import SingleRank
from pke.unsupervised import Kcore
from pke.unsupervised import LV
from pke.unsupervised import OutlierDetection

import os
from os import listdir

import string

import xml.etree.ElementTree as ET

import re

import json

import numpy as np

import networkx as nx

import time

import operator

import igraph

from keyword_extractor_kcore import KcoreKeywordExtractor


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
# Indicates whether we run the code on train or test data respectively
mode = 'train'
# 'train'
# 'test'

In [9]:
# create an extended stopword list that includes common adjectives, reporting verbs, determiners, functional words, etc.
stopwords_extend = []
with open('./stopwords_snowball_expanded.txt') as f:
    stopwords_extend = f.read().splitlines()
stopwords_list = stopwords_extend
stopwords_list += list(string.punctuation)
stopwords_list += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
stopwords_list += stopwords.words('english')
# stopwords_list_stemmed = []
# ps = SnowballStemmer('porter')
# for stopw in stopwords_list:
#     stopwords_list_stemmed.append(ps.stem(stopw))
# stopwords_list += stopwords_list_stemmed
stopwords_list.extend(['\t', '\n', '\x0b', '\x0c', '\r', ' '])
common_adjectives = []
with open('./common_adjectives.txt') as f:
    common_adjectives = f.read().splitlines()
print(common_adjectives)
reporting_verbs = []
with open('./reporting_verbs.txt') as f:
    reporting_verbs = f.read().splitlines()
print(reporting_verbs)
determiners = []
with open('./determiners.txt') as f:
    determiners = f.read().splitlines()
print(determiners)
functional_words = []
with open('./functional_words.txt') as f:
    functional_words = f.read().splitlines()
print(functional_words)

stopwords_with_capital = [stop.title() for stop in stopwords_list]
stopwords_list.extend(stopwords_with_capital)
stopwords_list = list(set(stopwords_list))
print(stopwords_list)

all_types_of_stopwords = list(set(common_adjectives + reporting_verbs + determiners + functional_words + stopwords_list))
all_types_of_stopwords_stemmed = []
ps = SnowballStemmer('porter')
for stopw in all_types_of_stopwords:
    all_types_of_stopwords_stemmed.append(ps.stem(stopw))
stoplist = all_types_of_stopwords + all_types_of_stopwords_stemmed
print(all_types_of_stopwords)

['other', 'new', 'good', 'great', 'big', 'small', 'large', 'different', 'little', 'important', 'bad', 'real', 'best', 'only', 'able', 'major', 'better', 'possible', 'true', 'easy', 'clear', 'recent', 'certain', 'difficult', 'available', 'likely', 'current', 'wrong', 'past', 'fine', 'common', 'poor', 'significant', 'similar', 'same', 'happy', 'serious', 'ready', 'various', 'entire', 'close', 'final', 'main', 'nice', 'huge', 'popular']
['admit', 'agree', 'believe', 'claim', 'confess', 'decide ', 'deny', 'doubt', 'explain', 'feel', 'hope', 'insist', 'mention', 'promise', 'reply', 'say', 'suggest', 'advise', 'prefer', 'propose', 'urge', 'accuse', 'apologise', 'prohibit', 'refuse', 'threaten', 'ask', 'allow', 'beg', 'encourage', 'forbid', 'instruct', 'invite', 'persuade', 'remind', 'tell', 'warn', 'want']
['the', 'a', 'an', 'this', 'that', 'these', 'those', 'my', 'your', 'his', 'her', 'its', 'our', 'their', 'few', 'little', 'much', 'many', 'lot', 'most', 'some', 'any', 'enough', 'all', 'bot

In [10]:
# dataset path
dataset_path = '/content/gdrive/MyDrive/Colab_notebooks/Official_GitHub_Repo_LV/ake-datasets/datasets/DUC-2001/'

# detection of dataset name by processing dataset path
dataset = dataset_path.split('/')[len(dataset_path.split('/'))-2]

In [11]:
# Load the ground truth of the corresponding dataset based on the mode (train/test sets)
keyphrases_folder = 'references/'

files_subfolder = ''
data_files = []
if mode == 'train':
  if dataset == 'NUS':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[:111]
    test_author_stem = 'test.author.stem.json'
    test_reader_stem = 'test.reader.stem.json'
    test_combined_stem = 'test.combined.stem.json'

    default_keyphrases_file = test_author_stem
  elif dataset == 'DUC-2001':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[:208]
    test_reader_stem = 'test.reader.stem.json'

    default_keyphrases_file = test_reader_stem
  elif dataset == 'SemEval-2010':
    files_subfolder = 'train/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    train_author_stem = 'train.author.stem.json'
    train_reader_stem = 'train.reader.stem.json'
    train_combined_stem = 'train.combined.stem.json'

    default_keyphrases_file = train_author_stem
  elif dataset == 'Inspec':
    files_subfolder = 'train/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    train_uncontr_stem = 'train.uncontr.stem.json'
    train_contr_stem = 'train.contr.stem.json'

    default_keyphrases_file = train_uncontr_stem
  elif dataset == '500N-KPCrowd':
    files_subfolder = 'train/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    train_reader_stem = 'train.reader.stem.json'

    default_keyphrases_file = train_reader_stem
  elif dataset == 'ACM':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[400:2304]
    test_author_stem = 'test.author.stem.json'
    
    default_keyphrases_file = test_author_stem
  elif dataset == 'PubMed':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[320:]
    test_author_stem = 'test.author.stem.json'
    
    default_keyphrases_file = test_author_stem
  elif dataset == 'Citeulike-180':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[80:]
    test_reader_stem = 'test.reader.stem.json'
    
    default_keyphrases_file = test_reader_stem
  elif dataset == 'CSTR':
    files_subfolder = 'train/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    train_author_stem = 'train.author.stem.json'
    
    default_keyphrases_file = train_author_stem
  elif dataset == 'WWW':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[330:1330]
    test_author_stem = 'test.author.stem.json'
    
    default_keyphrases_file = test_author_stem

elif mode == 'test':
  dataset = dataset_path.split('/')[len(dataset_path.split('/'))-2]
  print(dataset)
  if dataset == 'NUS':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    print(data_files)
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[111:211]
    test_author_stem = 'test.author.stem.json'
    test_reader_stem = 'test.reader.stem.json'
    test_combined_stem = 'test.combined.stem.json'

    default_keyphrases_file = test_author_stem
  elif dataset == 'DUC-2001':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[208:]
    test_reader_stem = 'test.reader.stem.json'

    default_keyphrases_file = test_reader_stem
  elif dataset == 'SemEval-2010':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    test_author_stem = 'test.author.stem.json'
    test_reader_stem = 'test.reader.stem.json'
    test_combined_stem = 'test.combined.stem.json'

    default_keyphrases_file = test_author_stem
  elif dataset == 'Inspec':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    test_uncontr_stem = 'test.uncontr.stem.json'
    test_contr_stem = 'test.contr.stem.json'

    default_keyphrases_file = test_uncontr_stem
  elif dataset == '500N-KPCrowd':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    test_reader_stem = 'test.reader.stem.json'

    default_keyphrases_file = test_reader_stem
  elif dataset == 'ACM':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[:400]
    test_author_stem = 'test.author.stem.json'

    default_keyphrases_file = test_author_stem
  elif dataset == 'PubMed':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[:320]
    test_author_stem = 'test.author.stem.json'
    
    default_keyphrases_file = test_author_stem
  elif dataset == 'Citeulike-180':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[:80]
    test_reader_stem = 'test.reader.stem.json'
    
    default_keyphrases_file = test_reader_stem
  elif dataset == 'CSTR':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    test_author_stem = 'test.author.stem.json'
    
    default_keyphrases_file = test_author_stem
  elif dataset == 'WWW':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    data_files = data_files[:330]
    test_author_stem = 'test.author.stem.json'
    
    default_keyphrases_file = test_author_stem
  elif dataset == 'KDD':
    files_subfolder = 'test/'
    data_files = [f for f in listdir(dataset_path+files_subfolder)]
    data_files = sorted(data_files)#, reverse=True)
    test_author_stem = 'test.author.stem.json'
    
    default_keyphrases_file = test_author_stem

print('Dataset:', dataset, 'mode:', mode, 'Number of files:', len(data_files), 'Sorted data file names:', data_files) 

Dataset: DUC-2001 mode: train Number of files: 208 Sorted data file names: ['AP830325-0143.xml', 'AP880217-0175.xml', 'AP880318-0051.xml', 'AP880330-0119.xml', 'AP880331-0140.xml', 'AP880409-0015.xml', 'AP880419-0131.xml', 'AP880510-0178.xml', 'AP880517-0226.xml', 'AP880520-0264.xml', 'AP880601-0040.xml', 'AP880613-0161.xml', 'AP880623-0135.xml', 'AP880629-0159.xml', 'AP880630-0295.xml', 'AP880705-0006.xml', 'AP880705-0018.xml', 'AP880705-0109.xml', 'AP880714-0142.xml', 'AP880801-0195.xml', 'AP880811-0299.xml', 'AP880816-0234.xml', 'AP880901-0052.xml', 'AP880902-0062.xml', 'AP880903-0092.xml', 'AP880913-0129.xml', 'AP880913-0204.xml', 'AP880914-0027.xml', 'AP880914-0079.xml', 'AP880926-0203.xml', 'AP880927-0089.xml', 'AP880927-0117.xml', 'AP880928-0054.xml', 'AP880928-0146.xml', 'AP881009-0072.xml', 'AP881017-0235.xml', 'AP881018-0136.xml', 'AP881126-0007.xml', 'AP881206-0114.xml', 'AP881210-0115.xml', 'AP881211-0027.xml', 'AP881216-0017.xml', 'AP881222-0089.xml', 'AP881222-0119.xml', 

In [12]:
# method to read document's text from xml file
def read_document_from_xml_file(input_file_path):
      title_clean_doc_text = ''
      # parse an xml file by name
      tree = ET.parse(input_file_path)
      root = tree.getroot()
      for document in root.findall('document'):
          for sentences in document.findall('sentences'):
              for sentence in sentences.findall('sentence'):
                  for tokens in sentence.findall('tokens'):
                      for token in tokens.findall('token'):
                          for word in token.findall('word'):
                              title_clean_doc_text += word.text + ' '
                  title_clean_doc_text += '\n'
      title_clean_doc_text = title_clean_doc_text.strip()
      return title_clean_doc_text

In [13]:
def document_gold_keyphrases(keyphrases_path, file_name): 
      keyphrases = []
      with open(keyphrases_path) as json_file:
        data = json.load(json_file)
      list_of_lists_keyphrases = data[file_name.replace('.xml', '').replace('.txt', '')]
      for list_keyphrase in list_of_lists_keyphrases:
        keyphrases.append(list_keyphrase[0].lower().translate(str.maketrans('', '', string.punctuation)).split())  
      return keyphrases

In [14]:
WSize = 20

# Opening a file 
file_results_kcore = open(dataset+'_'+mode+'_GoW_vs_AdjM_kcore_weightedVersion_WS'+str(WSize)+'.csv', 'w') 
file_results_if = open(dataset+'_'+mode+'_GoW_vs_AdjM_if_weightedVersion_WS'+str(WSize)+'.csv', 'w') 
file_results_ocsvm = open(dataset+'_'+mode+'_GoW_vs_AdjM_ocsvm_weightedVersion_WS'+str(WSize)+'.csv', 'w') 
file_results_lv = open(dataset+'_'+mode+'_GoW_vs_AdjM_lv_weightedVersion_WS'+str(WSize)+'.csv', 'w')
file_results_prw = open(dataset+'_'+mode+'_GoW_vs_AdjM_prw_weightedVersion_WS'+str(WSize)+'.csv', 'w') 
file_results_b = open(dataset+'_'+mode+'_GoW_vs_AdjM_bet_weightedVersion_WS'+str(WSize)+'.csv', 'w') 
file_results_n = open(dataset+'_'+mode+'_GoW_vs_AdjM_nd_weightedVersion_WS'+str(WSize)+'.csv', 'w') 

# Writing a string to file 
file_results_kcore.write('filename,P,R,F1,topNkcore,vocab\n') 
file_results_if.write('filename,Pkcore,Rkcore,F1kcore,Pif,Rif,F1if,P20,R20,F120,topNif,vocab\n') 
file_results_ocsvm.write('filename,Pkcore,Rkcore,F1kcore,Pif,Rif,F1if,P20,R20,F120\n') 
file_results_lv.write('filename,Pkcore,Rkcore,F1kcore,Pif,Rif,F1if,P20,R20,F120\n') 
file_results_prw.write('filename,Pkcore,Rkcore,F1kcore,Pif,Rif,F1if,P20,R20,F120\n') 
file_results_b.write('filename,Pkcore,Rkcore,F1kcore,Pif,Rif,F1if,P20,R20,F120\n')
file_results_n.write('filename,Pkcore,Rkcore,F1kcore,Pif,Rif,F1if,P20,R20,F120\n')

recall_list_kcore = []
precision_list_kcore = []
fm_list_kcore = []

recall_list_OD_IF = {}
precision_list_OD_IF = {}
fm_list_OD_IF = {}

recall_list_LV = {}
precision_list_LV = {}
fm_list_LV = {}

recall_list_OD_OCSVM = {}
precision_list_OD_OCSVM = {}
fm_list_OD_OCSVM = {}

recall_list_PR = {}
precision_list_PR = {}
fm_list_PR = {}

recall_list_b = {}
precision_list_b = {}
fm_list_b = {}

recall_list_n = {}
precision_list_n = {}
fm_list_n = {}

topN_list = ['kcore', 'if', 20]

for topN in topN_list:
  recall_list_OD_IF[topN] = []
  precision_list_OD_IF[topN] = []
  fm_list_OD_IF[topN] = []

  recall_list_OD_OCSVM[topN] = []
  precision_list_OD_OCSVM[topN] = []
  fm_list_OD_OCSVM[topN] = []

  recall_list_LV[topN] = []
  precision_list_LV[topN] = []
  fm_list_LV[topN] = []

  recall_list_PR[topN] = []
  precision_list_PR[topN] = []
  fm_list_PR[topN] = []

  recall_list_b[topN] = []
  precision_list_b[topN] = []
  fm_list_b[topN] = []

  recall_list_n[topN] = []
  precision_list_n[topN] = []
  fm_list_n[topN] = []

for fvc, filevc in enumerate(data_files):
  print(fvc, filevc)

  keyphrases = document_gold_keyphrases(dataset_path+keyphrases_folder+default_keyphrases_file, filevc)
  if len(keyphrases) == 0:# or filevc == '29080.xml':
    print('No keyphrase for this file...')
    continue  
  flatten_keywords = [word for sublist in keyphrases for word in sublist]
  flatten_keywords = list(set(flatten_keywords))
  print(len(flatten_keywords), flatten_keywords)

  doc_text = read_document_from_xml_file(dataset_path+files_subfolder+filevc)
  # print('Before removing syntax annotations:', doc_text[:300])
  syntax_annotations = ['-LRB-', '-RRB-', '-LCB-', '-RCB-', '-LSB-', '-RSB-']
  
  for sa in syntax_annotations:
    doc_text = doc_text.replace(sa, '')

  # print('After removing syntax annotations:', doc_text[:300])

  doc_text = doc_text.replace('\n', ' ')
  doc_text = re.sub("\s\s+", " ", doc_text)
  # print('After space normalization:', doc_text[:500])
  # define the set of valid Part-of-Speeches

#############################################k-core method#############################
  extractor_kcore = Kcore()
  # load the content of the document
  # the input language is set to English (used for the stoplist)
  # normalization is set to stemming (computed with Porter's stemming algorithm)
  extractor_kcore.load_document(input=doc_text, language="en", normalization='stemming')
  vocab_kcore, text = extractor_kcore.candidate_selection_all(stoplist)

  extractor_kw = KcoreKeywordExtractor(directed=False, weighted = True, window_size=WSize)
  results_kcore = extractor_kw.extract(vocab_kcore, ' '.join(text))

  results_kcore = list(dict(results_kcore).keys())

  # returns the k-core of the graph. In the graph, nodes are words of
  #    certain part-of-speech (nouns and adjectives) that are connected if
  #    they occur in a window of 10 words.
  # gof, num_different_words = extractor_kcore.build_weighted_word_graph_all(window=10)
  # results_kcore = extractor_kcore.weighted_k_core()

  # print(results_kcore)
  # number_of_kcores = len(results_kcore)
  # # print('number of various kcores:', number_of_kcores, 'vocab:', len(vocab_kcore.keys()))#, ', out of:', gof.number_of_nodes(),'=?', len(vocab_kcore.keys()), '(all num_different_words:', num_different_words, '), percentage:', number_of_kcore_nodes/gof.number_of_nodes(), ', percentage based on all words:', number_of_kcore_nodes/num_different_words)
  # dictionary_kcore_graphs = {}
  # for graph in results_kcore:
  #   vertices_degrees = []
  #   # print('number_of_graph_nodes:', graph.vcount())
  #   for v in graph.vs:
  #     # print('v:', v, 'degree:', graph.degree(v))
  #     vertices_degrees.append(graph.degree(v))
  #   if len(vertices_degrees) > 0 and min(vertices_degrees) not in dictionary_kcore_graphs.keys():
  #     dictionary_kcore_graphs[min(vertices_degrees)] = [graph]
  #   elif len(vertices_degrees) > 0:
  #     dictionary_kcore_graphs[min(vertices_degrees)].append(graph)  
  # sorted_kcore_graphs = sorted(dictionary_kcore_graphs.items(), key=operator.itemgetter(0))

  # # for k in sorted_kcore_graphs:
  # #   print(k[0], len(k[1]))

  # print('Main core selection:', sorted_kcore_graphs[len(sorted_kcore_graphs)-1][0], len(sorted_kcore_graphs[len(sorted_kcore_graphs)-1][1]))

  number_of_kcore_nodes = len(results_kcore)#sorted_kcore_graphs[len(sorted_kcore_graphs)-1][1][0].vcount()
  print('number_of_kcore_nodes', number_of_kcore_nodes)
#############################################end of k-core method######################

#############################################IF method#############################
  extractor_OD_IF = OutlierDetection()
  # load the content of the document
  # the input language is set to English (used for the stoplist)
  # normalization is set to stemming (computed with Porter's stemming algorithm)
  extractor_OD_IF.load_document(input=doc_text, tagger=False, language="en", normalization='stemming')
  # select unigrams as candidates.
  vocab = extractor_OD_IF.candidate_selection(stoplist)
  # calculate co-occurrences between each pair of candidate words in a window of 10 words in corporating positional info
  extractor_OD_IF.find_cooccurrences(stoplist, window=WSize, position=False)
  # convert co-occurrences dictionary to a co-occurrrence numpy matrix
  extractor_OD_IF.build_cooccurrences_matrix()
  # weight the candidates using the decision function of the IF outlier detector
  # nu = number_of_kcore_nodes/len(vocab.keys())
  nu = number_of_kcore_nodes/len(vocab.keys())
  if nu > 0.5:
    nu = 0.49
  extractor_OD_IF.candidate_weighting(nu = nu, max_samples = 'auto', method = 'IF')
  # get the number_of_kcore_nodes-highest scored candidates as keywords
  results_OD_IF_scores_topkcore = extractor_OD_IF.get_n_best(n=number_of_kcore_nodes, stemming=True)
  results_OD_IF_topkcore = []
  for u in results_OD_IF_scores_topkcore:
      results_OD_IF_topkcore.append(u[0])
  print('number_of_OD_IF_nodes_topkcore:', len(results_OD_IF_topkcore), ', out of:', len(vocab.keys()), ', percentage:', len(results_OD_IF_topkcore)/len(vocab.keys()))

  # nu = 'auto'
  number_of_if_outliers = extractor_OD_IF.candidate_weighting(nu = 'auto', max_samples = 'auto', method = 'IF')
  # get the number_of_kcore_nodes-highest scored candidates as keywords
  results_OD_IF_scores_topif = extractor_OD_IF.get_n_best(n=number_of_if_outliers, stemming=True)
  results_OD_IF_topif = []
  for u in results_OD_IF_scores_topif:
      results_OD_IF_topif.append(u[0])
  print('number_of_OD_IF_nodes_topif:', len(results_OD_IF_topif), ', out of:', len(vocab.keys()), ', percentage:', len(results_OD_IF_topif)/len(vocab.keys()))

  if dataset == 'NUS' or dataset == 'ACM' or dataset == 'SemEval-2010':
    nu = 0.05
  elif dataset == 'DUC-2001':
    nu = 0.1
  elif dataset == 'KDD' or dataset == 'WWW':
    nu = 0.2
  # nu = 0.05 for full-text papers, 0.1 for news articles, 0.2 for abstracts
  top20_if_outliers = extractor_OD_IF.candidate_weighting(nu = nu, max_samples = 'auto', method = 'IF')
  # get the number_of_kcore_nodes-highest scored candidates as keywords
  results_OD_IF_scores_top20 = extractor_OD_IF.get_n_best(n=20, stemming=True)
  results_OD_IF_top20 = []
  for u in results_OD_IF_scores_top20:
      results_OD_IF_top20.append(u[0])
  print('number_of_OD_IF_nodes_top20:', len(results_OD_IF_top20), ', out of:', len(vocab.keys()), ', percentage:', len(results_OD_IF_top20)/len(vocab.keys()))

#############################################end of IF method######################

#############################################OCSVM method#############################
  extractor_OD_OCSVM = OutlierDetection()
  # load the content of the document
  # the input language is set to English (used for the stoplist)
  # normalization is set to stemming (computed with Porter's stemming algorithm)
  extractor_OD_OCSVM.load_document(input=doc_text, tagger=False, language="en", normalization='stemming')
  # select unigrams as candidates.
  vocab = extractor_OD_OCSVM.candidate_selection(stoplist)
  # calculate co-occurrences between each pair of candidate words in a window of 10 words in corporating positional info
  extractor_OD_OCSVM.find_cooccurrences(stoplist, window=WSize, position=False)
  # convert co-occurrences dictionary to a co-occurrrence numpy matrix
  extractor_OD_OCSVM.build_cooccurrences_matrix()
  # weight the candidates using the decision function of the OCSVM outlier detector
  # nu = number_of_kcore_nodes/len(vocab.keys())
  nu = number_of_kcore_nodes/len(vocab.keys())
  if nu > 0.5:
    nu = 0.49
  extractor_OD_OCSVM.candidate_weighting(nu = nu, method = 'OCSVM')
  # get the number_of_kcore_nodes-highest scored candidates as keywords
  results_OD_OCSVM_scores_topkcore = extractor_OD_OCSVM.get_n_best(n=number_of_kcore_nodes, stemming=True)
  results_OD_OCSVM_topkcore = []
  for u in results_OD_OCSVM_scores_topkcore:
      results_OD_OCSVM_topkcore.append(u[0])
  print('number_of_OD_OCSVM_nodes:', len(results_OD_OCSVM_topkcore), ', out of:', len(vocab.keys()), ', percentage:', len(results_OD_OCSVM_topkcore)/len(vocab.keys()))

  # nu = 0.05 for full-text papers, 0.1 for news articles, 0.2 for abstracts
  if dataset == 'NUS' or dataset == 'ACM' or dataset == 'SemEval-2010':
    nu = 0.05
  elif dataset == 'DUC-2001':
    nu = 0.1
  elif dataset == 'KDD' or dataset == 'WWW':
    nu = 0.2
  extractor_OD_OCSVM.candidate_weighting(nu = nu, method = 'OCSVM')
  # get the number_of_kcore_nodes-highest scored candidates as keywords
  results_OD_OCSVM_scores_topif = extractor_OD_OCSVM.get_n_best(n=number_of_if_outliers, stemming=True)
  results_OD_OCSVM_topif = []
  for u in results_OD_OCSVM_scores_topif:
      results_OD_OCSVM_topif.append(u[0])
  print('number_of_OD_OCSVM_topif:', len(results_OD_OCSVM_topif), ', out of:', len(vocab.keys()), ', percentage:', len(results_OD_OCSVM_topif)/len(vocab.keys()))

  # nu = 0.05 for full-text papers, 0.1 for news articles, 0.2 for abstracts
  if dataset == 'NUS' or dataset == 'ACM' or dataset == 'SemEval-2010':
    nu = 0.05
  elif dataset == 'DUC-2001':
    nu = 0.1
  elif dataset == 'KDD' or dataset == 'WWW':
    nu = 0.2
  extractor_OD_OCSVM.candidate_weighting(nu = nu, method = 'OCSVM')
  # get the number_of_kcore_nodes-highest scored candidates as keywords
  results_OD_OCSVM_scores_top20 = extractor_OD_OCSVM.get_n_best(n=20, stemming=True)
  results_OD_OCSVM_top20 = []
  for u in results_OD_OCSVM_scores_top20:
      results_OD_OCSVM_top20.append(u[0])
  print('number_of_OD_OCSVM_top20:', len(results_OD_OCSVM_top20), ', out of:', len(vocab.keys()), ', percentage:', len(results_OD_OCSVM_top20)/len(vocab.keys()))
#############################################end of OCSVM method######################

#############################################LV method#############################
  extractor_LV = LV()
  # load the content of the document
  # the input language is set to English (used for the stoplist)
  # normalization is set to stemming (computed with Porter's stemming algorithm)
  extractor_LV.load_document(input=doc_text, tagger=False, language="en", normalization='stemming')
  # select unigrams as candidates.
  vocab = extractor_LV.candidate_selection(stoplist)
  # calculate co-occurrences between each pair of candidate words in a window of 10 words in corporating positional info
  extractor_LV.find_cooccurrences(stoplist, window=WSize, position=False)
  # convert co-occurrences dictionary to a co-occurrrence numpy matrix
  extractor_LV.build_cooccurrences_matrix()
  # weight the candidates using a the euclidean distance from the mean vector
  extractor_LV.candidate_weighting()
  # get the number_of_kcore_nodes-highest scored candidates as keywords
  results_LV_scores_topkcore = extractor_LV.get_n_best(n=number_of_kcore_nodes, stemming=True)
  results_LV_topkcore = []
  for u in results_LV_scores_topkcore:
      results_LV_topkcore.append(u[0])
  print('number_of_LV_topkcore:', len(results_LV_topkcore), ', out of:', len(vocab.keys()), ', percentage:', len(results_LV_topkcore)/len(vocab.keys()))

  # get the number_of_if_outliers-highest scored candidates as keywords
  results_LV_scores_topif = extractor_LV.get_n_best(n=number_of_if_outliers, stemming=True)
  results_LV_topif = []
  for u in results_LV_scores_topif:
      results_LV_topif.append(u[0])
  print('number_of_LV_topif:', len(results_LV_topif), ', out of:', len(vocab.keys()), ', percentage:', len(results_LV_topif)/len(vocab.keys()))

  # get the top20-highest scored candidates as keywords
  results_LV_scores_top20 = extractor_LV.get_n_best(n=20, stemming=True)
  results_LV_top20 = []
  for u in results_LV_scores_top20:
      results_LV_top20.append(u[0])
  print('number_of_LV_top20:', len(results_LV_top20), ', out of:', len(vocab.keys()), ', percentage:', len(results_LV_top20)/len(vocab.keys()))

#############################################end of LV method######################

#############################################PR method#############################
  extractor_PR = SingleRank()
  # load the content of the document
  # the input language is set to English (used for the stoplist)
  # normalization is set to stemming (computed with Porter's stemming algorithm)
  extractor_PR.load_document(input=doc_text, language="en", normalization='stemming')
  vocab_PR = extractor_PR.candidate_selection_all(stoplist)
  # select the longest sequences of nouns and adjectives as candidates.
  # candidates = extractor_PR.ngram_selection(n=1, pos=pos)
  # weight the candidates using the sum of their word's scores that are
  #  computed using random walk. In the graph, nodes are words of
  #  certain part-of-speech (nouns and adjectives) that are connected if
  #  they occur in a window of 10 words.
  results_PR_scores = extractor_PR.candidate_weighting(window=WSize)
  results_PR_scores_topkcore = results_PR_scores[:number_of_kcore_nodes]
  # get the number_of_kcore_nodes-highest scored candidates as keywords
  results_PR_topkcore = []
  for u in results_PR_scores_topkcore:
      results_PR_topkcore.append(u[0])
  print('number_of_PR_topkcore:', len(results_PR_topkcore), ', out of:', len(vocab_PR.keys()),  ', percentage:', len(results_PR_topkcore)/len(vocab_PR.keys()))

  results_PR_scores_topif = results_PR_scores[:number_of_if_outliers]
  # get the number_of_if_outliers-highest scored candidates as keywords
  results_PR_topif = []
  for u in results_PR_scores_topif:
      results_PR_topif.append(u[0])
  print('number_of_PR_topif:', len(results_PR_topif), ', out of:', len(vocab_PR.keys()),  ', percentage:', len(results_PR_topif)/len(vocab_PR.keys()))


  results_PR_scores_top20 =results_PR_scores[:20]
  # get the top20-highest scored candidates as keywords
  results_PR_top20 = []
  for u in results_PR_scores_top20:
      results_PR_top20.append(u[0])
  print('number_of_PR_top20:', len(results_PR_top20), ', out of:', len(vocab_PR.keys()),  ', percentage:', len(results_PR_top20)/len(vocab_PR.keys()))
#############################################end of PR method######################

#############################################B method#############################
  extractor_b = Betweeness()
  # load the content of the document
  # the input language is set to English (used for the stoplist)
  # normalization is set to stemming (computed with Porter's stemming algorithm)
  extractor_b.load_document(input=doc_text, language="en", normalization='stemming')
  vocab_b = extractor_b.candidate_selection_all(stoplist)
  results_b_scores = extractor_b.candidate_weighting(window=WSize)
  results_b_scores_topkcore = results_b_scores[:number_of_kcore_nodes]
  # get the number_of_kcore_nodes-highest scored candidates as keywords
  results_b_topkcore = []
  for u in results_b_scores_topkcore:
      results_b_topkcore.append(u[0])
  print('number_of_b_topkcore:', len(results_b_topkcore), ', out of:', len(vocab_b.keys()),  ', percentage:', len(results_b_topkcore)/len(vocab_b.keys()))

  results_b_scores_topif = results_b_scores[:number_of_if_outliers]
  # get the number_of_if_outliers-highest scored candidates as keywords
  results_b_topif = []
  for u in results_b_scores_topif:
      results_b_topif.append(u[0])
  print('number_of_b_topif:', len(results_b_topif), ', out of:', len(vocab_b.keys()),  ', percentage:', len(results_b_topif)/len(vocab_b.keys()))


  results_b_scores_top20 = results_b_scores[:20]
  # get the top20-highest scored candidates as keywords
  results_b_top20 = []
  for u in results_b_scores_top20:
      results_b_top20.append(u[0])
  print('number_of_b_top20:', len(results_b_top20), ', out of:', len(vocab_b.keys()),  ', percentage:', len(results_b_top20)/len(vocab_b.keys()))
#############################################end of B method######################

#############################################N method#############################
  extractor_n = NodeDegree()
  # load the content of the document
  # the input language is set to English (used for the stoplist)
  # normalization is set to stemming (computed with Porter's stemming algorithm)
  extractor_n.load_document(input=doc_text, language="en", normalization='stemming')
  vocab_n = extractor_n.candidate_selection_all(stoplist)
  results_n_scores = extractor_n.candidate_weighting(window=WSize)
  results_n_scores_topkcore = results_n_scores[:number_of_kcore_nodes]
  # get the number_of_kcore_nodes-highest scored candidates as keywords
  results_n_topkcore = []
  for u in results_n_scores_topkcore:
      results_n_topkcore.append(u[0])
  print('number_of_n_topkcore:', len(results_n_topkcore), ', out of:', len(vocab_n.keys()),  ', percentage:', len(results_n_topkcore)/len(vocab_n.keys()))

  results_n_scores_topif = results_n_scores[:number_of_if_outliers]
  # get the number_of_if_outliers-highest scored candidates as keywords
  results_n_topif = []
  for u in results_n_scores_topif:
      results_n_topif.append(u[0])
  print('number_of_n_topif:', len(results_n_topif), ', out of:', len(vocab_n.keys()),  ', percentage:', len(results_n_topif)/len(vocab_n.keys()))


  results_n_scores_top20 = results_n_scores[:20]
  # get the top20-highest scored candidates as keywords
  results_n_top20 = []
  for u in results_n_scores_top20:
      results_n_top20.append(u[0])
  print('number_of_n_top20:', len(results_n_top20), ', out of:', len(vocab_n.keys()),  ', percentage:', len(results_n_top20)/len(vocab_n.keys()))
#############################################end of N method######################


  print('----------Current--------------')

  recall_kcore = len(set(results_kcore).intersection(flatten_keywords))/float(len(flatten_keywords))
  precision_kcore = 0.0
  if len(results_kcore) > 0: 
    precision_kcore = len(set(results_kcore).intersection(flatten_keywords))/float(len(results_kcore))
  fm_kcore = 0.0
  if recall_kcore > 0 or precision_kcore > 0:
    fm_kcore = 2.0*(precision_kcore*recall_kcore)/(precision_kcore+recall_kcore)
  file_results_kcore.write(filevc+','+str(precision_kcore)+','+str(recall_kcore)+','+str(fm_kcore)+','+str(number_of_kcore_nodes)+','+str(len(vocab_kcore.keys()))+'\n') 
  file_results_kcore.flush()

  # k_core_vertices = []
  # for v in sorted_kcore_graphs[len(sorted_kcore_graphs)-1][1][0].vs:
  #   k_core_vertices.append(v['name'])
  # # print(k_core_vertices)
  # # break
  # recall_kcore = len(set(k_core_vertices).intersection(flatten_keywords))/float(len(flatten_keywords))
  # precision_kcore = 0.0
  # if len(k_core_vertices) > 0: 
  #   precision_kcore = len(set(k_core_vertices).intersection(flatten_keywords))/float(len(k_core_vertices))
  # fm_kcore = 0.0
  # if recall_kcore > 0 or precision_kcore > 0:
  #   fm_kcore = 2.0*(precision_kcore*recall_kcore)/(precision_kcore+recall_kcore)
  # file_results_kcore.write(filevc+','+str(precision_kcore)+','+str(recall_kcore)+','+str(fm_kcore)+','+str(number_of_kcore_nodes)+','+str(len(vocab_kcore.keys()))+'\n') 
  # file_results_kcore.flush()

  recall_OD_IF_topkcore = len(set(results_OD_IF_topkcore).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_OD_IF_topif = len(set(results_OD_IF_topif).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_OD_IF_top20 = len(set(results_OD_IF_top20).intersection(flatten_keywords))/float(len(flatten_keywords))
  precision_OD_IF_topkcore = 0.0
  if len(results_OD_IF_topkcore) > 0: 
    precision_OD_IF_topkcore = len(set(results_OD_IF_topkcore).intersection(flatten_keywords))/float(len(results_OD_IF_topkcore))
  precision_OD_IF_topif = 0.0
  if len(results_OD_IF_topif) > 0: 
    precision_OD_IF_topif = len(set(results_OD_IF_topif).intersection(flatten_keywords))/float(len(results_OD_IF_topif))
  precision_OD_IF_top20 = 0.0
  if len(results_OD_IF_top20) > 0: 
    precision_OD_IF_top20 = len(set(results_OD_IF_top20).intersection(flatten_keywords))/float(len(results_OD_IF_top20))
  fm_OD_IF_topkcore = 0.0
  if recall_OD_IF_topkcore > 0 or precision_OD_IF_topkcore > 0:
    fm_OD_IF_topkcore = 2.0*(precision_OD_IF_topkcore*recall_OD_IF_topkcore)/(precision_OD_IF_topkcore+recall_OD_IF_topkcore)
  fm_OD_IF_topif = 0.0
  if recall_OD_IF_topif > 0 or precision_OD_IF_topif > 0:
    fm_OD_IF_topif = 2.0*(precision_OD_IF_topif*recall_OD_IF_topif)/(precision_OD_IF_topif+recall_OD_IF_topif)
  fm_OD_IF_top20 = 0.0
  if recall_OD_IF_top20 > 0 or precision_OD_IF_top20 > 0:
    fm_OD_IF_top20 = 2.0*(precision_OD_IF_top20*recall_OD_IF_top20)/(precision_OD_IF_top20+recall_OD_IF_top20)
  file_results_if.write(filevc+','+str(precision_OD_IF_topkcore)+','+str(recall_OD_IF_topkcore)+','+str(fm_OD_IF_topkcore)+','+str(precision_OD_IF_topif)+','+str(recall_OD_IF_topif)+','+str(fm_OD_IF_topif)+','+str(precision_OD_IF_top20)+','+str(recall_OD_IF_top20)+','+str(fm_OD_IF_top20)+','+str(number_of_if_outliers)+','+str(len(vocab.keys()))+'\n') 
  file_results_if.flush()

  recall_OD_OCSVM_topkcore = len(set(results_OD_OCSVM_topkcore).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_OD_OCSVM_topif = len(set(results_OD_OCSVM_topif).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_OD_OCSVM_top20 = len(set(results_OD_OCSVM_top20).intersection(flatten_keywords))/float(len(flatten_keywords))
  precision_OD_OCSVM_topkcore = 0.0
  if len(results_OD_OCSVM_topkcore) > 0: 
    precision_OD_OCSVM_topkcore = len(set(results_OD_OCSVM_topkcore).intersection(flatten_keywords))/float(len(results_OD_OCSVM_topkcore))
  precision_OD_OCSVM_topif = 0.0
  if len(results_OD_OCSVM_topif) > 0: 
    precision_OD_OCSVM_topif = len(set(results_OD_OCSVM_topif).intersection(flatten_keywords))/float(len(results_OD_OCSVM_topif))
  precision_OD_OCSVM_top20 = 0.0
  if len(results_OD_OCSVM_top20) > 0: 
    precision_OD_OCSVM_top20 = len(set(results_OD_OCSVM_top20).intersection(flatten_keywords))/float(len(results_OD_OCSVM_top20))
  fm_OD_OCSVM_topkcore = 0.0
  if recall_OD_OCSVM_topkcore > 0 or precision_OD_OCSVM_topkcore > 0:
    fm_OD_OCSVM_topkcore = 2.0*(precision_OD_OCSVM_topkcore*recall_OD_OCSVM_topkcore)/(precision_OD_OCSVM_topkcore+recall_OD_OCSVM_topkcore)
  fm_OD_OCSVM_topif = 0.0
  if recall_OD_OCSVM_topif > 0 or precision_OD_OCSVM_topif > 0:
    fm_OD_OCSVM_topif = 2.0*(precision_OD_OCSVM_topif*recall_OD_OCSVM_topif)/(precision_OD_OCSVM_topif+recall_OD_OCSVM_topif)
  fm_OD_OCSVM_top20 = 0.0
  if recall_OD_OCSVM_top20 > 0 or precision_OD_OCSVM_top20 > 0:
    fm_OD_OCSVM_top20 = 2.0*(precision_OD_OCSVM_top20*recall_OD_OCSVM_top20)/(precision_OD_OCSVM_top20+recall_OD_OCSVM_top20)
  file_results_ocsvm.write(filevc+','+str(precision_OD_OCSVM_topkcore)+','+str(recall_OD_OCSVM_topkcore)+','+str(fm_OD_OCSVM_topkcore)+','+str(precision_OD_OCSVM_topif)+','+str(recall_OD_OCSVM_topif)+','+str(fm_OD_OCSVM_topif)+','+str(precision_OD_OCSVM_top20)+','+str(recall_OD_OCSVM_top20)+','+str(fm_OD_OCSVM_top20)+'\n') 
  file_results_ocsvm.flush()

  recall_LV_topkcore = len(set(results_LV_topkcore).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_LV_topif = len(set(results_LV_topif).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_LV_top20 = len(set(results_LV_top20).intersection(flatten_keywords))/float(len(flatten_keywords))
  precision_LV_topkcore = 0.0
  if len(results_LV_topkcore) > 0: 
    precision_LV_topkcore = len(set(results_LV_topkcore).intersection(flatten_keywords))/float(len(results_LV_topkcore))
  precision_LV_topif = 0.0
  if len(results_LV_topif) > 0: 
    precision_LV_topif = len(set(results_LV_topif).intersection(flatten_keywords))/float(len(results_LV_topif))
  precision_LV_top20 = 0.0
  if len(results_LV_top20) > 0: 
    precision_LV_top20 = len(set(results_LV_top20).intersection(flatten_keywords))/float(len(results_LV_top20))
  fm_LV_topkcore = 0.0
  if recall_LV_topkcore > 0 or precision_LV_topkcore > 0:
    fm_LV_topkcore = 2.0*(precision_LV_topkcore*recall_LV_topkcore)/(precision_LV_topkcore+recall_LV_topkcore)
  fm_LV_topif = 0.0
  if recall_LV_topif > 0 or precision_LV_topif > 0:
    fm_LV_topif = 2.0*(precision_LV_topif*recall_LV_topif)/(precision_LV_topif+recall_LV_topif)
  fm_LV_top20 = 0.0
  if recall_LV_top20 > 0 or precision_LV_top20 > 0:
    fm_LV_top20 = 2.0*(precision_LV_top20*recall_LV_top20)/(precision_LV_top20+recall_LV_top20)
  file_results_lv.write(filevc+','+str(precision_LV_topkcore)+','+str(recall_LV_topkcore)+','+str(fm_LV_topkcore)+','+str(precision_LV_topif)+','+str(recall_LV_topif)+','+str(fm_LV_topif)+','+str(precision_LV_top20)+','+str(recall_LV_top20)+','+str(fm_LV_top20)+'\n') 
  file_results_lv.flush()

  recall_PR_topkcore = len(set(results_PR_topkcore).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_PR_topif = len(set(results_PR_topif).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_PR_top20 = len(set(results_PR_top20).intersection(flatten_keywords))/float(len(flatten_keywords))
  precision_PR_topkcore = 0.0
  if len(results_PR_topkcore) > 0: 
    precision_PR_topkcore = len(set(results_PR_topkcore).intersection(flatten_keywords))/float(len(results_PR_topkcore))
  precision_PR_topif = 0.0
  if len(results_PR_topif) > 0: 
    precision_PR_topif = len(set(results_PR_topif).intersection(flatten_keywords))/float(len(results_PR_topif))
  precision_PR_top20 = 0.0
  if len(results_PR_top20) > 0: 
    precision_PR_top20 = len(set(results_PR_top20).intersection(flatten_keywords))/float(len(results_PR_top20))
  fm_PR_topkcore = 0.0
  if recall_PR_topkcore > 0 or precision_PR_topkcore > 0:
    fm_PR_topkcore = 2.0*(precision_PR_topkcore*recall_PR_topkcore)/(precision_PR_topkcore+recall_PR_topkcore)
  fm_PR_topif = 0.0
  if recall_PR_topif > 0 or precision_PR_topif > 0:
    fm_PR_topif = 2.0*(precision_PR_topif*recall_PR_topif)/(precision_PR_topif+recall_PR_topif)
  fm_PR_top20 = 0.0
  if recall_PR_top20 > 0 or precision_PR_top20 > 0:
    fm_PR_top20 = 2.0*(precision_PR_top20*recall_PR_top20)/(precision_PR_top20+recall_PR_top20)
  file_results_prw.write(filevc+','+str(precision_PR_topkcore)+','+str(recall_PR_topkcore)+','+str(fm_PR_topkcore)+','+str(precision_PR_topif)+','+str(recall_PR_topif)+','+str(fm_PR_topif)+','+str(precision_PR_top20)+','+str(recall_PR_top20)+','+str(fm_PR_top20)+'\n') 
  file_results_prw.flush()
  
  recall_b_topkcore = len(set(results_b_topkcore).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_b_topif = len(set(results_b_topif).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_b_top20 = len(set(results_b_top20).intersection(flatten_keywords))/float(len(flatten_keywords))
  precision_b_topkcore = 0.0
  if len(results_b_topkcore) > 0: 
    precision_b_topkcore = len(set(results_b_topkcore).intersection(flatten_keywords))/float(len(results_b_topkcore))
  precision_b_topif = 0.0
  if len(results_b_topif) > 0: 
    precision_b_topif = len(set(results_b_topif).intersection(flatten_keywords))/float(len(results_b_topif))
  precision_b_top20 = 0.0
  if len(results_b_top20) > 0: 
    precision_b_top20 = len(set(results_b_top20).intersection(flatten_keywords))/float(len(results_b_top20))
  fm_b_topkcore = 0.0
  if recall_b_topkcore > 0 or precision_b_topkcore > 0:
    fm_b_topkcore = 2.0*(precision_b_topkcore*recall_b_topkcore)/(precision_b_topkcore+recall_b_topkcore)
  fm_b_topif = 0.0
  if recall_b_topif > 0 or precision_b_topif > 0:
    fm_b_topif = 2.0*(precision_b_topif*recall_b_topif)/(precision_b_topif+recall_b_topif)
  fm_b_top20 = 0.0
  if recall_b_top20 > 0 or precision_b_top20 > 0:
    fm_b_top20 = 2.0*(precision_b_top20*recall_b_top20)/(precision_b_top20+recall_b_top20)
  file_results_b.write(filevc+','+str(precision_b_topkcore)+','+str(recall_b_topkcore)+','+str(fm_b_topkcore)+','+str(precision_b_topif)+','+str(recall_b_topif)+','+str(fm_b_topif)+','+str(precision_b_top20)+','+str(recall_b_top20)+','+str(fm_b_top20)+'\n') 
  file_results_b.flush()

  recall_n_topkcore = len(set(results_n_topkcore).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_n_topif = len(set(results_n_topif).intersection(flatten_keywords))/float(len(flatten_keywords))
  recall_n_top20 = len(set(results_n_top20).intersection(flatten_keywords))/float(len(flatten_keywords))
  precision_n_topkcore = 0.0
  if len(results_n_topkcore) > 0: 
    precision_n_topkcore = len(set(results_n_topkcore).intersection(flatten_keywords))/float(len(results_n_topkcore))
  precision_n_topif = 0.0
  if len(results_n_topif) > 0: 
    precision_n_topif = len(set(results_n_topif).intersection(flatten_keywords))/float(len(results_n_topif))
  precision_n_top20 = 0.0
  if len(results_n_top20) > 0: 
    precision_n_top20 = len(set(results_n_top20).intersection(flatten_keywords))/float(len(results_n_top20))
  fm_n_topkcore = 0.0
  if recall_n_topkcore > 0 or precision_n_topkcore > 0:
    fm_n_topkcore = 2.0*(precision_n_topkcore*recall_n_topkcore)/(precision_n_topkcore+recall_n_topkcore)
  fm_n_topif = 0.0
  if recall_n_topif > 0 or precision_n_topif > 0:
    fm_n_topif = 2.0*(precision_n_topif*recall_n_topif)/(precision_n_topif+recall_n_topif)
  fm_n_top20 = 0.0
  if recall_n_top20 > 0 or precision_n_top20 > 0:
    fm_n_top20 = 2.0*(precision_n_top20*recall_n_top20)/(precision_n_top20+recall_n_top20)
  file_results_n.write(filevc+','+str(precision_n_topkcore)+','+str(recall_n_topkcore)+','+str(fm_n_topkcore)+','+str(precision_n_topif)+','+str(recall_n_topif)+','+str(fm_n_topif)+','+str(precision_n_top20)+','+str(recall_n_top20)+','+str(fm_n_top20)+'\n') 
  file_results_n.flush()


  print('recall_b:', recall_b_top20, 'recall_n:', recall_n_top20, 'recall_PR:', recall_PR_top20, ', recall_OD_OCSVM:', recall_OD_OCSVM_top20, ', recall_OD_IF:', recall_OD_IF_top20, ', recall_LV:', recall_LV_top20)
  print('precision_b:', precision_b_top20, 'precision_n:', precision_n_top20, 'precision_PR:', precision_PR_top20, ', precision_OD_OCSVM:', precision_OD_OCSVM_top20, ', precision_OD_IF:', precision_OD_IF_top20, ', precision_LV:', precision_LV_top20)
  print('fm_b:', fm_b_top20, 'fm_n:', fm_n_top20, 'fm_PR:', fm_PR_top20, ', fm_OD_OCSVM:', fm_OD_OCSVM_top20, ', fm_OD_IF:', fm_OD_IF_top20, ', fm_LV:', fm_LV_top20)


  print('--------------AVER----------------')

  recall_list_kcore.append(recall_kcore)
  precision_list_kcore.append(precision_kcore)
  fm_list_kcore.append(fm_kcore)

  recall_list_OD_IF['kcore'].append(recall_OD_IF_topkcore)
  precision_list_OD_IF['kcore'].append(precision_OD_IF_topkcore)
  fm_list_OD_IF['kcore'].append(fm_OD_IF_topkcore)

  recall_list_OD_IF['if'].append(recall_OD_IF_topif)
  precision_list_OD_IF['if'].append(precision_OD_IF_topif)
  fm_list_OD_IF['if'].append(fm_OD_IF_topif)

  recall_list_OD_IF[20].append(recall_OD_IF_top20)
  precision_list_OD_IF[20].append(precision_OD_IF_top20)
  fm_list_OD_IF[20].append(fm_OD_IF_top20)

  recall_list_OD_OCSVM['kcore'].append(recall_OD_OCSVM_topkcore)
  precision_list_OD_OCSVM['kcore'].append(precision_OD_OCSVM_topkcore)
  fm_list_OD_OCSVM['kcore'].append(fm_OD_OCSVM_topkcore)

  recall_list_OD_OCSVM['if'].append(recall_OD_OCSVM_topif)
  precision_list_OD_OCSVM['if'].append(precision_OD_OCSVM_topif)
  fm_list_OD_OCSVM['if'].append(fm_OD_OCSVM_topif)

  recall_list_OD_OCSVM[20].append(recall_OD_OCSVM_top20)
  precision_list_OD_OCSVM[20].append(precision_OD_OCSVM_top20)
  fm_list_OD_OCSVM[20].append(fm_OD_OCSVM_top20)

  recall_list_LV['kcore'].append(recall_LV_topkcore)
  precision_list_LV['kcore'].append(precision_LV_topkcore)
  fm_list_LV['kcore'].append(fm_LV_topkcore)

  recall_list_LV['if'].append(recall_LV_topif)
  precision_list_LV['if'].append(precision_LV_topif)
  fm_list_LV['if'].append(fm_LV_topif)

  recall_list_LV[20].append(recall_LV_top20)
  precision_list_LV[20].append(precision_LV_top20)
  fm_list_LV[20].append(fm_LV_top20)

  recall_list_PR['kcore'].append(recall_PR_topkcore)
  precision_list_PR['kcore'].append(precision_PR_topkcore)
  fm_list_PR['kcore'].append(fm_PR_topkcore)

  recall_list_PR['if'].append(recall_PR_topif)
  precision_list_PR['if'].append(precision_PR_topif)
  fm_list_PR['if'].append(fm_PR_topif)

  recall_list_PR[20].append(recall_PR_top20)
  precision_list_PR[20].append(precision_PR_top20)
  fm_list_PR[20].append(fm_PR_top20)

  recall_list_b['kcore'].append(recall_b_topkcore)
  precision_list_b['kcore'].append(precision_b_topkcore)
  fm_list_b['kcore'].append(fm_b_topkcore)

  recall_list_b['if'].append(recall_b_topif)
  precision_list_b['if'].append(precision_b_topif)
  fm_list_b['if'].append(fm_b_topif)

  recall_list_b[20].append(recall_b_top20)
  precision_list_b[20].append(precision_b_top20)
  fm_list_b[20].append(fm_b_top20)

  recall_list_n['kcore'].append(recall_n_topkcore)
  precision_list_n['kcore'].append(precision_n_topkcore)
  fm_list_n['kcore'].append(fm_n_topkcore)

  recall_list_n['if'].append(recall_n_topif)
  precision_list_n['if'].append(precision_n_topif)
  fm_list_n['if'].append(fm_n_topif)

  recall_list_n[20].append(recall_n_top20)
  precision_list_n[20].append(precision_n_top20)
  fm_list_n[20].append(fm_n_top20)

  print('fm_list_kcore:', sum(fm_list_kcore)/len(fm_list_kcore), 'fm_b_av:', sum(fm_list_b['kcore'])/len(fm_list_b['kcore']), 'fm_n_av:', sum(fm_list_n['kcore'])/len(fm_list_n['kcore']), 'fm_PR_av:', sum(fm_list_PR['kcore'])/len(fm_list_PR['kcore']), 'fm_OD_OCSVM_av:', sum(fm_list_OD_OCSVM['kcore'])/len(fm_list_OD_OCSVM['kcore']), 'fm_OD_IF_av:', sum(fm_list_OD_IF['kcore'])/len(fm_list_OD_IF['kcore']), 'fm_LV_av:', sum(fm_list_LV['kcore'])/len(fm_list_LV['kcore']))
  print()

  print('recall_b_av:', sum( recall_list_b[20])/len( recall_list_b[20]), 'recall_n_av:', sum( recall_list_n[20])/len( recall_list_n[20]), 'recall_PR_av:', sum( recall_list_PR[20])/len( recall_list_PR[20]), ', recall_OD_OCSVM_av:', sum(recall_list_OD_OCSVM[20])/len(recall_list_OD_OCSVM[20]), ', recall_OD_IF_av:', sum(recall_list_OD_IF[20])/len(recall_list_OD_IF[20]), ', recall_LV_av:', sum(recall_list_LV[20])/len(recall_list_LV[20]))
  print('precision_b_av:', sum(precision_list_b[20])/len(precision_list_b[20]), 'precision_n_av:', sum(precision_list_n[20])/len(precision_list_n[20]), 'precision_PR_av:', sum(precision_list_PR[20])/len(precision_list_PR[20]), 'precision_OD_OCSVM_av:', sum(precision_list_OD_OCSVM[20])/len(precision_list_OD_OCSVM[20]), 'precision_OD_IF_av:', sum(precision_list_OD_IF[20])/len(precision_list_OD_IF[20]), 'precision_LV_av:', sum(precision_list_LV[20])/len(precision_list_LV[20]))
  print('fm_b_av:', sum(fm_list_b[20])/len(fm_list_b[20]), 'fm_n_av:', sum(fm_list_n[20])/len(fm_list_n[20]), 'fm_PR_av:', sum(fm_list_PR[20])/len(fm_list_PR[20]), 'fm_OD_OCSVM_av:', sum(fm_list_OD_OCSVM[20])/len(fm_list_OD_OCSVM[20]), 'fm_OD_IF_av:', sum(fm_list_OD_IF[20])/len(fm_list_OD_IF[20]), 'fm_LV_av:', sum(fm_list_LV[20])/len(fm_list_LV[20]))

  print('')

  # if fvc==5:
  #     break
# Closing file 
file_results_kcore.close() 
file_results_if.close() 
file_results_ocsvm.close()
file_results_lv.close()
file_results_prw.close() 
file_results_b.close()
file_results_n.close()

0 AP830325-0143.xml
15 ['environment', 'crude', 'alaska', 'oil', 'equip', 'major', 'us', '987foot', 'valdez', 'exxon', 'tanker', 'catastroph', 'cleanup', 'effort', 'spill']
number_of_kcore_nodes 15
self.word_to_sentID: {'million': 0, 'gallon': 0, 'crude': 0, 'oil': 0, 'spill': 0, 'tanker': 0, 'ran': 0, 'aground': 0, 'spread': 0, 'across': 0, 'wildlif': 0, 'rich': 0, 'stretch': 0, 'ocean': 0, 'saturday': 0, 'alaska': 0, 'chief': 0, 'environment': 0, 'offic': 0, 'critic': 0, 'cleanup': 0, 'effort': 0, 'slow': 0, 'biggest': 1, 'histori': 1, 'creat': 1, 'slick': 1, 'seven': 1, 'mile': 1, 'wide': 1, 'princ': 1, 'william': 1, 'sound': 1, 'coast': 2, 'guard': 2, 'reef': 2, 'island': 2, 'western': 2, 'edg': 2, 'bligh': 2, 'touch': 2, 'situat': 3, 'think': 3, 'everyon': 3, 'secret': 3, 'nightmar': 3, 'happen': 3, 'traffic': 3, 'denni': 3, 'kelso': 3, 'commission': 3, 'depart': 3, 'conserv': 3, 'barrel': 4, 'north': 4, 'slope': 4, 'earli': 4, 'friday': 4, '987foot': 4, 'exxon': 4, 'valdez': 4, '

inliers: (97,) i.e. 11
outliers: (11,)
number_of_OD_IF_nodes_top20: 11 , out of: 108 , percentage: 0.10185185185185185


self.word_to_sentID: {'hurrican': 0, 'expert': 0, 'predict': 0, 'turbul': 0, 'summer': 0, 'atlant': 0, 'ocean': 0, 'fiercer': 0, 'storm': 0, 'swirl': 0, 'sea': 0, 'imposs': 0, 'know': 0, 'popul': 0, 'area': 0, 'william': 1, 'gray': 1, 'professor': 1, 'atmospher': 1, 'scienc': 1, 'colorado': 1, 'state': 1, 'univers': 1, 'friday': 1, 'expect': 1, 'six': 1, 'year': 1, 'averag': 1, 'last': 1, 'decad': 1, 'form': 2, 'rel': 2, 'season': 2, 'brought': 3, 'seven': 3, 'spot': 3, 'offici': 4, 'begin': 4, 'june': 4, 'activ': 4, 'period': 4, 'aug': 4, 'wind': 5, 'air': 5, 'pressur': 5, 'pattern': 5, 'annual': 5, 'forecast': 5, 'issu': 5, 'formal': 5, 'late': 5, 'earli': 6, 'outlook': 6, '10th': 6, 'nation': 6, 'confer': 6, 'anticip': 7, 'intens': 7, 'paper': 7, 'present': 7, 'model': 8, 'equat': 8, 'el': 8, 'nino': 8, 'eastern': 8, 'pacif': 8, 'water': 8, 'warmer': 8, 'base': 9, 'presenc': 9, 'light': 9, 'easterli': 9, 'approach': 9, 'end': 9, 'actual': 10, 'occur': 10, 'money': 11, 'origin': 11, 

inliers: (116,) i.e. 13
outliers: (13,)
number_of_OD_IF_nodes_top20: 13 , out of: 129 , percentage: 0.10077519379844961
self.word_to_sentID: {'yasser': 0, 'arafat': 0, 'tuesday': 0, 'unit': 0, 'state': 0, 'kill': 0, 'plo': 0, 'offici': 0, 'palestinian': 0, 'guerrilla': 0, 'attack': 0, 'american': 0, 'target': 0, 'depart': 2, 'washington': 2, 'receiv': 2, 'report': 2, 'alleg': 2, 'involv': 2, 'assassin': 2, 'khalil': 2, 'wazir': 2, 'command': 2, 'slain': 3, 'april': 3, 'raid': 3, 'hous': 3, 'tuni': 3, 'tunisia': 3, 'isra': 4, 'spoke': 4, 'condit': 4, 'identifi': 4, 'squad': 4, 'carri': 4, 'knew': 5, 'approv': 5, 'plan': 5, 'slay': 5, 'palestin': 6, 'liber': 6, 'organ': 6, 'leader': 6, 'threat': 6, 'govern': 6, 'document': 6, 'obtain': 6, 'arab': 6, 'assist': 8, 'secretari': 8, 'richard': 8, 'murphi': 8, 'spokesman': 9, 'charl': 9, 'redman': 9, 'touch': 9, 'number': 9, 'middl': 9, 'eastern': 9, 'countri': 9, 'citizen': 9, 'facil': 9, 'ad': 10, 'interpret': 10, 'contact': 10, 'foundat': 1

inliers: (60,) i.e. 57
outliers: (57,)
number_of_OD_IF_nodes_topkcore: 57 , out of: 117 , percentage: 0.48717948717948717
inliers: (111,) i.e. 6
outliers: (6,)
number_of_OD_IF_nodes_topif: 6 , out of: 117 , percentage: 0.05128205128205128
inliers: (105,) i.e. 12
outliers: (12,)
number_of_OD_IF_nodes_top20: 20 , out of: 117 , percentage: 0.17094017094017094


...

self.word_to_sentID: {'patienc': 0, 'muster': 0, 'run': 0, 'mile': 0, 'marathon': 0, 'tendenc': 1, 'finish': 1, 'race': 1, 'soon': 1, 'head': 1, 'start': 1, 'recov': 1, 'virtu': 2, 'beach': 2, 'sunday': 2, 'men': 2, 'women': 2, 'winner': 2, 'shrewdli': 2, 'wait': 2, 'mistak': 2, 'sun': 2, 'obscur': 2, 'cloud': 2, 'cover': 2, 'earli': 2, 'burn': 2, 'toll': 2, 'runner': 2, 'rex': 3, 'wilson': 3, 'zealand': 3, 'patient': 3, 'lead': 3, 'pack': 3, 'sort': 3, 'took': 3, 'roll': 3, 'victori': 3, 'hour': 3, 'minut': 3, 'cours': 3, 'record': 3, 'set': 4, 'ric': 4, 'sayr': 4, 'play': 5, 'calm': 5, 'orderli': 5, 'fashion': 5, 'wen': 6, 'yanmin': 6, '23year': 6, 'student': 6, 'beje': 6, 'china': 6, 'pick': 6, 'strung': 6, 'kick': 6, 'laura': 6, 'konatz': 6, 'toronto': 6, 'win': 6, 'broke': 7, 'last': 7, 'year': 7, 'diann': 7, 'rodger': 7, 'robert': 8, 'molinatti': 8, 'huntington': 8, 'wheelchair': 8, 'divis': 8, 'within': 9, 'pacif': 9, 'rim': 9, 'consist': 9, 'team': 9, 'countri': 9, 'total': 10,

inliers: (52,) i.e. 6
outliers: (6,)
number_of_OD_IF_nodes_top20: 9 , out of: 58 , percentage: 0.15517241379310345


self.word_to_sentID: {'pneumonia': 0, 'stricken': 0, 'actress': 0, 'elizabeth': 0, 'taylor': 0, 'suffer': 0, 'complic': 0, 'fifth': 0, 'week': 0, 'hospit': 0, 'includ': 0, 'infect': 0, 'doctor': 0, 'six': 0, 'recoveri': 1, 'miss': 1, 'death': 1, 'viral': 1, 'bacteri': 1, 'yeast': 1, 'today': 1, 'resolv': 2, 'newli': 2, 'acquir': 2, 'secondari': 2, 'stay': 2, 'st': 2, 'john': 2, 'health': 2, 'center': 2, 'santa': 2, 'monica': 2, 'publicist': 2, 'chen': 2, 'sam': 2, 'improv': 3, 'significantli': 3, 'physician': 3, 'develop': 4, 'fungemia': 4, 'candida': 4, 'albican': 4, 'blood': 4, 'commonli': 4, 'refer': 4, 'approxim': 4, 'intraven': 4, 'therapi': 4, 'ad': 4, 'elabor': 4, 'gener': 5, 'consid': 5, 'initi': 5, 'contract': 5, 'antibiot': 5, 'effect': 5, 'bacteria': 5, 'virus': 5}
inliers: (46,) i.e. 12
outliers: (12,)
number_of_OD_OCSVM_nodes: 6 , out of: 58 , percentage: 0.10344827586206896
inliers: (45,) i.e. 13
outliers: (13,)
number_of_OD_OCSVM_topif: 9 , out of: 58 , percentage: 0.155

self.word_to_sentID: {'prolong': 0, 'drought': 0, 'bark': 0, 'beetl': 0, 'rare': 0, 'fungu': 0, 'dri': 0, 'kill': 0, 'brush': 0, 'tree': 0, 'surround': 0, 'southern': 0, 'california': 0, 'neighborhood': 0, 'creat': 0, 'potenti': 0, 'worst': 0, 'wildfir': 0, 'season': 0, 'decad': 0, 'fire': 0, 'offici': 0, 'monday': 0, 'map': 1, 'releas': 1, 'lo': 1, 'angel': 1, 'counti': 1, 'depart': 1, 'show': 1, 'vulner': 1, 'zone': 1, 'found': 1, 'summer': 1, 'chaparr': 1, 'cover': 1, 'foothil': 1, 'mountain': 1, 'canyon': 1, 'slope': 1, 'dens': 1, 'develop': 1, 'residenti': 1, 'district': 1, 'suburb': 1, 'state': 2, 'forestri': 2, 'open': 2, 'region': 2, 'mean': 2, 'firefight': 2, 'crew': 2, 'place': 2, 'higher': 2, 'alert': 2, 'prevent': 2, 'regul': 2, 'effect': 2, 'announc': 3, 'came': 3, 'week': 3, 'earlier': 3, 'full': 3, 'month': 3, 'schedul': 3, 'heavili': 3, 'wood': 3, 'terrain': 3, 'nation': 3, 'forest': 3, 'last': 4, 'realli': 4, 'rainfal': 4, 'nt': 4, 'recuper': 4, 'seven': 4, 'year': 4, 

inliers: (149,) i.e. 17
outliers: (17,)
number_of_OD_IF_nodes_top20: 17 , out of: 166 , percentage: 0.10240963855421686
self.word_to_sentID: {'mad': 0, 'cow': 0, 'diseas': 0, 'enigmat': 0, 'nervou': 0, 'disord': 0, 'kill': 0, 'thousand': 0, 'cattl': 0, 'britain': 0, 'trade': 0, 'friction': 0, 'europ': 0, '37billion': 0, 'british': 0, 'beef': 0, 'industri': 0, 'friday': 1, 'west': 1, 'germani': 1, 'join': 1, 'franc': 1, 'ban': 1, 'cite': 1, 'health': 1, 'fear': 1, 'relat': 1, 'mysteri': 1, 'ailment': 1, 'technic': 1, 'name': 1, 'bovin': 1, 'spongiform': 1, 'encephalopathi': 1, 'bse': 1, 'concern': 2, 'transmit': 2, 'human': 2, 'subject': 2, 'topic': 2, 'pub': 2, 'press': 2, 'crimp': 2, 'domest': 2, 'sale': 2, 'assur': 3, 'govern': 3, 'safe': 3, 'consum': 3, 'scientist': 3, 'remain': 3, 'skeptic': 3, 'offici': 4, 'complain': 4, 'french': 4, 'german': 4, 'result': 4, 'econom': 4, 'price': 5, 'drop': 5, 'wake': 5, 'scare': 5, 'impos': 6, 'protect': 6, 'total': 7, 'unjustifi': 7, 'illeg': 7